Create a Python script to perform a sentiment analysis of the Twitter activity of various news outlets, and to present your findings visually.

--> Your final output should provide a visualized summary of the sentiments expressed in Tweets sent out by the following news organizations: **BBC, CBS, CNN, Fox, and New York times**.

---------------------------------------------------------------------------------------------------------
1. The first plot will be and/or feature the following:

* Be a scatter plot of sentiments of the last **100** tweets sent out by each news organization, ranging from -1.0 to 1.0, where a score of 0 expresses a neutral sentiment, -1 the most negative sentiment possible, and +1 the most positive sentiment possible.
* Each plot point will reflect the _compound_ sentiment of a tweet.
* Sort each plot point by its relative timestamp.

--------------------------------------------------------------------------------------------------------
2. The second plot will be a bar plot visualizing the _overall_ sentiments of the last 100 tweets from each organization. For this plot, you will again aggregate the compound sentiments analyzed by VADER.

--------------------------------------------------------------------------------------------------------

The tools of the trade you will need for your task as a data analyst include the following: tweepy, pandas, matplotlib, and VADER.

--------------------------------------------------------------------------------------------------------
Your final Jupyter notebook must:

* Pull last 100 tweets from each outlet.
* Perform a sentiment analysis with the compound, positive, neutral, and negative scoring for each tweet.
* Pull into a DataFrame the tweet's source account, its text, its date, and its compound, positive, neutral, and negative sentiment scores.
* Export the data in the DataFrame into a CSV file.
* Save PNG images for each plot.

--------------------------------------------------------------------------------------------------------
As final considerations:

* You must complete your analysis using a Jupyter notebook.
* You must use the Matplotlib or Pandas plotting libraries.
* Include a written description of THREE observable trends based on the data.
* Include proper labeling of your plots, including plot titles (with date of analysis) and axes labels.


In [2]:
#Import Dependencies
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
import tweepy
from pprint import pprint

In [ ]:
#Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#Import the keys
from config import (consumer_key, consumer_secret,
                   access_token, access_token_secret)

# Twitter API Keys
#consumer_key = "lQDyO9LHsHV7rw12KbBGP2H4S"
#consumer_secret = "SrjdhJcNqJUsnMckBdp9vICDSXS8udXvEJr1iZQKKgMmyp4rsS"
#access_token = "1007989024773074944-XZqI1XqtOjXdGDZlEMlftuNkRQbCVe"
#access_token_secret = "58vmGsdNdKv7dXmc1NfMs7VfnbGarNgelr4IGnQf6bLjj"

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
def getNewsSentiment(search='@bbc', tweet_count=1):
    #print(f"getNewsSentiment, {search} {tweet_count}")
    api = tweepy.API(auth)
    # Variables for holding sentiments
    #tweet_source_list = []
    sentiment_list = []
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    tweets_ago = []    
    
    count = 0
        
    for result in tweepy.Cursor(api.user_timeline, search).items(tweet_count):
        sentiment = analyzer.polarity_scores(result._json['text'])
        #pprint(result._json)
        #tweet_source_list.append(result._json['user']['name'])
        sentiment_list.append(result._json['text'])
        compound_list.append(sentiment['compound'])
        positive_list.append(sentiment['pos'])
        negative_list.append(sentiment['neg'])
        neutral_list.append(sentiment['neu'])
        tweets_ago.append(count)
        count += 1
    
    return pd.DataFrame({'Tweet Source':search,
                 'Tweet':sentiment_list,
                 'Compound':compound_list,
                 'Positive':positive_list, 
                 'Negative':negative_list, 
                 'Neutral':neutral_list,
                 'Tweets Ago':tweets_ago})

In [ ]:
# Target Search Term
news_stations = ("@BBC", "@CBS", "@CNN", "@Fox", "@nytimes")

In [ ]:
getNewsSentiment("@nytimes", tweet_count=2)

In [ ]:
tweet_count = 100
station_feedbacks = []
for station in news_stations:
    #print(station)
    station_feedbacks.append(getNewsSentiment(station, tweet_count))
    #print(df)
    #station_feedback_df = df = pd.DataFrame.append(station_feedback_df, df, ignore_index=True)

len(station_feedbacks)

In [ ]:
count = len(station_feedbacks)
merged_df = pd.DataFrame()
for i in range(count):
    merged_df = pd.DataFrame.append(merged_df, station_feedbacks[i],ignore_index=True)

In [ ]:
Saving the Sentiments into a csv file

In [ ]:
#Saving the csv
media_sentiment_filename = f"NewsMood_{str(time.strftime('%m-%d-%y'))}_Sentiments.csv"
merged_df.to_csv(media_sentiment_filename)

In [ ]:
station_feedbacks[4].head()

In [ ]:
Sentiment Scatter Plot¶

In [ ]:
colors = ['lightcoral', 'green', 'red', 'blue', 'yellow']
count = 0

for station in station_feedbacks:
    plt.scatter(station['Tweets Ago'], 
                station['Compound'],
                c=colors[count],
                edgecolor="black", linewidths=1, marker="o",
                alpha=1, label=station['Tweet Source'][0].split('@')[1])

    count += 1

now = datetime.now()
now = now.strftime("%Y-%m-%d %H:%M")
plt.title(f"Sentiment Analysis of Media Tweets ({now})")
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")

plt.grid(True)

plt.legend(loc="upper left", bbox_to_anchor=(1,1), title="Media Sources")

# Saving the img
media_sentiment_plot = f"NewsMood_{str(time.strftime('%m-%d-%y'))}_Sentiment_Scatter_Plot.png"
plt.savefig(media_sentiment_plot, bbox_inches='tight')

plt.show()

In [ ]:
Sentiment Bar Plot

In [ ]:
# Group by tweet source and take the average of the compound
merged_df_gb = merged_df.groupby(['Tweet Source']).mean()['Compound']

av_merged_df = pd.DataFrame(merged_df_gb)

In [ ]:
merged_df_gb

In [ ]:
av_merged_df = av_merged_df.reset_index()

In [ ]:
av_merged_df

In [ ]:
y_axis = av_merged_df['Compound']
#x_axis = av_merged_df['Tweet Source']
x_axis = ["BBC", "CBS", "CNN", "Fox", "NY Times"]

plt.bar(x_axis, y_axis, 
        color=['lightcoral', 'green', 'red', 'blue', 'yellow'], alpha=0.5, align="center")

now = datetime.now()
now = now.strftime("%Y-%m-%d %H:%M")
plt.title(f"Overall Media Sentiment based on Twitter ({now})")
plt.ylabel("Tweet Polarity")

# Saving the img
av_media_sentiment_bar_plot = f"NewsMood_{str(time.strftime('%m-%d-%y'))}_Sentiment_Bar_Plot.png"
plt.savefig(av_media_sentiment_bar_plot)

plt.show()